In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
dataset = dataset.drop("User ID", axis=1)

In [4]:
dataset= pd.get_dummies(dataset, dtype=int, drop_first=True)
dataset


,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [5]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [6]:
dataset.columns

Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [7]:
independent = dataset[['Age', 'EstimatedSalary', 'Gender_Male']]

In [8]:
dependent = dataset[["Purchased"]]

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(independent, dependent,random_state=0, test_size=0.30)

In [10]:
# from sklearn.preprocessing import StandardScaler
# scx= StandardScaler()
# x_train = scx.fit_transform(x_train)
# x_test = scx.transform(x_test)

# scy = StandardScaler()
# y_train = scy.fit_transform(y_train)
# y_test= scy.transform(y_test)

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

param_grid = {
    "solver":['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'],
    "penalty": ['l2']
}

grid = GridSearchCV(LogisticRegression(), param_grid, verbose=3, refit=True, n_jobs=-1, scoring = 'f1_weighted')

grid.fit(x_train, y_train)


Fitting 5 folds for each of 6 candidates, totalling 30 fits


C:\Anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Anaconda3\Lib\site-packages\scipy\optimize\_linesearch.py:313: LineSearchWarning: The line search algorithm did not converge
  alpha_star, phi_star, old_fval, derphi_star = scalar_search_wolfe2(
C:\Anaconda3\Lib\site-packages\sklearn\utils\optimize.py:74: LineSearchWarning: The line search algorithm did not converge
  ret = line_search_wolfe2(


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg',
                                    'newton-cholesky', 'sag', 'saga']},
             scoring='f1_weighted', verbose=3)

In [13]:
y_pred = grid.predict(x_test)

In [27]:
from sklearn.metrics import f1_score
f1_macro = f1_score(y_test, y_pred, average='weighted')
print("f1_macro score for the best parameter{}:".format(grid.best_params_), f1_macro )

f1_macro score for the best parameter{'penalty': 'l2', 'solver': 'newton-cg'}: 0.8906190214115365


In [16]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

cm = confusion_matrix(y_test, y_pred)
clf_report = classification_report(y_test, y_pred)

print(cm)
print(clf_report)

[[74  5]
 [ 8 33]]
              precision    recall  f1-score   support

           0       0.90      0.94      0.92        79
           1       0.87      0.80      0.84        41

    accuracy                           0.89       120
   macro avg       0.89      0.87      0.88       120
weighted avg       0.89      0.89      0.89       120



In [23]:
from sklearn.metrics import roc_auc_score
roc_score = roc_auc_score(y_test, grid.predict_proba(x_test)[:,1])
print("roc_auc_score is {}".format(roc_score))

roc_auc_score is 0.9484408768138315


In [17]:
re = grid.cv_results_
table = pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.062629,0.005034,0.019733,0.003135,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.816207,0.775048,0.644599,0.927778,0.874356,0.807598,0.096542,3
1,0.011157,0.006333,0.015166,0.002688,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.503106,0.503106,0.591398,0.480769,0.520202,0.519716,0.037966,4
2,0.214573,0.055672,0.014610,0.004604,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.835985,0.802399,0.644599,0.927778,0.909115,0.823975,0.100806,1
3,0.085509,0.059564,0.014797,0.003689,l2,newton-cholesky,"{'penalty': 'l2', 'solver': 'newton-cholesky'}",0.835985,0.802399,0.644599,0.927778,0.909115,0.823975,0.100806,1
4,0.040936,0.015713,0.017589,0.002069,l2,sag,"{'penalty': 'l2', 'solver': 'sag'}",0.503106,0.503106,0.503106,0.480769,0.480769,0.494171,0.010943,5
5,0.012273,0.002143,0.016124,0.004584,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.503106,0.503106,0.503106,0.480769,0.480769,0.494171,0.010943,5


f1_macro score for the best parameter{'penalty': 'l2', 'solver': 'newton-cg'}: 0.8906190214115365
